In [1]:
import requests
import sqlite3
from lxml import etree
import settings

In [3]:
con = sqlite3.connect(settings.db_location)
c = con.cursor()
r = c.execute("SELECT * FROM sources;")

for row in r:
    if row[3] == 1:
        print('Pull!!! -> {connect_point}'.format(connect_point=row[2]))
        url = row[2]

In [5]:
tree = etree.fromstring(requests.get(url).text)

In [6]:
articles = tree.findall('.//item')

In [39]:
for article in articles:
    article_details = article.getchildren()

In [11]:
articles[0].find('link').text

'https://www.nytimes.com/2019/10/24/opinion/trump-economy.html'

In [40]:
article_data = {}
for item in article_details:
    article_data[item.tag] = item.text

In [41]:
article_data

{'title': 'Republicans Only Pretend to Be Patriots',
 'link': 'https://www.nytimes.com/2019/09/23/opinion/democrats-republicans.html',
 'description': 'And Democrats need to expose them for what they are.',
 'author': 'By Paul Krugman',
 'pubDate': 'Mon, 23 Sep 2019 22:00:06 GMT'}

In [14]:
class seek(object):
    def __init__(self, db_location=settings.db_location):
        self.db_location = db_location
        
    def _connect_to_db(self):
        """Returns connection to package sqlite db, which holds all source urls 
        as well as articles which have been previously uplaoded to Instapaper by the application"""
        con = sqlite3.connect(self.db_location)
        return con
    
    def _gather_sources(self, connection):
        """Gathers active sources from the package db which will be referenced
        by the application when seeking newly published articles"""
        source_query = """SELECT name, connect_point, type FROM sources WHERE status=1;"""
    
        c = con.cursor()
        
        source_dict = {}
        for row in c.execute(source_query):
            name, connect_point, _type = row
            if _type in source_dict.keys():
                soruce_dict[_type].append((name,connect_point))
            else:
                source_dict[_type] = [(name,connect_point)]
        return source_dict

    def _rss_gather(self, source_name, rss_feed_link):
        """Pull all articles given a rss feed link
        Details that are passed back will mirror those required for insertion into the articles table
        (name, source, connect_point, publish_time)"""
        tree = etree.fromstring(requests.get(rss_feed_link).text)
        articles = tree.findall('.//item')

        articles_detials = []
        status = 0
        _type = 'RSS'
        for article in articles:
            articles_details.append((article.find('title').text, source_name,
                                     article.find('link').text, article.find('pubDate').text))
        return articles_detials
                        
    def check_sources(self):
        con = _connect_to_db()
        source_dict = _gather_sources(connection=con)
        article_details = []
        if 'RSS' in source_dict.keys():
            for source in source_dict['RSS']:
                source_name, rss_feed_link = source
                article_details.append(self._rss_gather(source_name=source_name,
                                                        rss_feed_link=rss_feed_link))
        elif 'Medium' in source_dict.keys():
            for source in source_dict['Medium']:
                source_name, medium_feed_link = source
                article_details.append(self._medium_gather(source_name=source_name,
                                                           medium_link=medium_link))
        
        con.close()